In [1]:
import os

os.chdir('../')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data_train_path = "Dataset/data_train.csv"

URM_all_dataframe = pd.read_csv(data_train_path)
print(URM_all_dataframe.head(10))

   user_id  item_id  data
0        0        0   1.0
1        0        2   1.0
2        0      120   1.0
3        0      128   1.0
4        0      211   1.0
5        0      232   1.0
6        0      282   1.0
7        0      453   1.0
8        0      458   1.0
9        0      491   1.0


In [3]:
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout



URM_all = sps.coo_matrix((URM_all_dataframe['data'], (URM_all_dataframe['user_id'], URM_all_dataframe['item_id'])))
URM_all = URM_all.tocsr()

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions


In [5]:
from Recommenders.SLIM.SLIMElasticNetRecommender_MultithreadWorking import MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

slimNet_W_sparse = sps.load_npz("SimilarityMatrixes/slim_best_W_TRAINVAL.npz")

recommender_rp3 = RP3betaRecommender(URM_train_validation)
recommender_rp3.fit(topK=14, alpha=0.3683550822991944, beta=0.19877125816137325, implicit=True)

alpha = 0.7414857378820595

similarity_matrix = slimNet_W_sparse.multiply(alpha)+recommender_rp3.W_sparse.multiply(1-alpha)

RP3betaRecommender: Similarity column 38121 (100.0%), 3614.46 column/sec. Elapsed time 10.55 sec


In [6]:
# Let's load our best model and make some predictions
from Recommenders.Hybrid.GeneralizedLinearCoupleHybridRecommender import GeneralizedLinearCoupleHybridRecommender
from Recommenders.MatrixFactorization.ImplicitALSRecommender import ImplicitALSRecommender

als_recommender = ImplicitALSRecommender(URM_train_validation)
als_recommender.fit(factors=1903, regularization=0.43781149051372426, iterations=10, use_gpu=False, **{"alpha": 10.365656056877015})

/Users/lucapagano/miniconda3/envs/RecSysPD/lib/python3.8/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 8 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/10 [00:00<?, ?it/s]

In [7]:
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender
similarity_recommender = ItemKNNCustomSimilarityRecommender(URM_train_validation)
similarity_recommender.fit(similarity_matrix)

recommenders = [similarity_recommender, als_recommender]

In [8]:
def objective_function(trial):
    alpha = trial.suggest_uniform('alpha', 0.0, 1.0)

    recommender = GeneralizedLinearCoupleHybridRecommender(URM_train_validation, recommenders)
    recommender.fit(alpha)
    
    result_dict, _ = evaluator_test.evaluateRecommender(recommender)
    return result_dict.loc[10]["MAP"]

class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame(columns = ["result"])
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        self.results_df = pd.concat([self.results_df, pd.DataFrame([hyperparam_dict])], ignore_index=True)
        self.results_df.to_csv("logs/Hybrid/hybrid_slim_rp3_IALS_NOEASERSUCA.csv", index = False)
        
        
import optuna

optuna_study = optuna.create_study(study_name="hybrid_slim_rp3_IALS", direction="maximize")
        
save_results = SaveResults()
        
optuna_study.optimize(objective_function,
                      callbacks=[save_results],
                      n_trials = 100
                      )

[I 2025-01-06 22:35:18,173] A new study created in memory with name: hybrid_slim_rp3_IALS
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_30079/2257660771.py:2: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform('alpha', 0.0, 1.0)


EvaluatorHoldout: Processed 35595 (100.0%) in 34.71 sec. Users per second: 1026


[I 2025-01-06 22:35:52,907] Trial 0 finished with value: 0.06033534560989091 and parameters: {'alpha': 0.5726711478953503}. Best is trial 0 with value: 0.06033534560989091.
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_30079/2257660771.py:2: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform('alpha', 0.0, 1.0)


EvaluatorHoldout: Processed 35595 (100.0%) in 35.89 sec. Users per second: 992


[I 2025-01-06 22:36:28,819] Trial 1 finished with value: 0.0607240349433758 and parameters: {'alpha': 0.9990296806718744}. Best is trial 1 with value: 0.0607240349433758.
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_30079/2257660771.py:2: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform('alpha', 0.0, 1.0)


...

[I 2025-01-06 23:32:21,338] Trial 97 finished with value: 0.0612023123900479 and parameters: {'alpha': 0.9326547826954337}. Best is trial 9 with value: 0.0615608766613803.
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_30079/2257660771.py:2: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform('alpha', 0.0, 1.0)


EvaluatorHoldout: Processed 35595 (100.0%) in 34.81 sec. Users per second: 1023


[I 2025-01-06 23:32:56,175] Trial 98 finished with value: 0.061548621283975306 and parameters: {'alpha': 0.7991985915465918}. Best is trial 9 with value: 0.0615608766613803.
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_30079/2257660771.py:2: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform('alpha', 0.0, 1.0)


EvaluatorHoldout: Processed 35595 (100.0%) in 33.98 sec. Users per second: 1047


[I 2025-01-06 23:33:30,186] Trial 99 finished with value: 0.052617791200385046 and parameters: {'alpha': 0.1668654038347226}. Best is trial 9 with value: 0.0615608766613803.


In [11]:
optuna_study.best_params

{'alpha': 0.7976651540039968}

# Let's fit all

In [ ]:
from Recommenders.SLIM.SLIMElasticNetRecommender_MultithreadWorking import MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

slimNet_W_sparse = sps.load_npz("SimilarityMatrixes/slim_best_W.npz")

recommender_rp3 = RP3betaRecommender(URM_all)
recommender_rp3.fit(topK=14, alpha=0.3683550822991944, beta=0.19877125816137325, implicit=True)

alpha = 0.7414857378820595

similarity_matrix = slimNet_W_sparse.multiply(alpha)+recommender_rp3.W_sparse.multiply(1-alpha)
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender
similarity_recommender = ItemKNNCustomSimilarityRecommender(URM_all)
similarity_recommender.fit(similarity_matrix)
recommenders = [similarity_recommender, als_recommender]

alpha_linear = 0.7976651540039968

recommenders_all = [similarity_recommender, als_recommender]
recommender = GeneralizedLinearCoupleHybridRecommender(URM_all, recommenders)
recommender.fit(alpha_linear)

RP3betaRecommender: Similarity column 38121 (100.0%), 3202.63 column/sec. Elapsed time 11.90 sec


/Users/lucapagano/miniconda3/envs/RecSysPD/lib/python3.8/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 8 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/10 [00:00<?, ?it/s]

In [6]:
target_user = pd.read_csv("Dataset/data_target_users_test.csv")
print(target_user.head(10))

user_id = target_user['user_id']
recommendations = []
for user in user_id:
    recommendations.append(recommender.recommend(user, cutoff=10))

# Saving in a submission file for the competition
submission = pd.DataFrame(columns=['user_id', 'item_list'])
submission['user_id'] = user_id
# Convert list of recommendations into a space-separated string
submission['item_list'] = [' '.join(map(str, recs)) for recs in recommendations]

# Save the submission file
submission.to_csv(f"submissions/hybrid_best_noEASER.csv", index=False)

   user_id
0        0
1        1
2        2
3        3
4        4
5        5
6        6
7        7
8        8
9        9


# EASER?

In [4]:
from Recommenders.SLIM.SLIMElasticNetRecommender_MultithreadWorking import MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

slimNet_W_sparse = sps.load_npz("SimilarityMatrixes/slim_best_W_TRAINVAL.npz")

recommender_rp3 = RP3betaRecommender(URM_train_validation)
recommender_rp3.fit(topK=14, alpha=0.3683550822991944, beta=0.19877125816137325, implicit=True)

alpha = 0.7414857378820595

similarity_matrix = slimNet_W_sparse.multiply(alpha)+recommender_rp3.W_sparse.multiply(1-alpha)

from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender
similarity_recommender = ItemKNNCustomSimilarityRecommender(URM_train_validation)
similarity_recommender.fit(similarity_matrix)

# Let's load our best model and make some predictions
from Recommenders.Hybrid.GeneralizedLinearCoupleHybridRecommender import GeneralizedLinearCoupleHybridRecommender
from Recommenders.MatrixFactorization.ImplicitALSRecommender import ImplicitALSRecommender

als_recommender = ImplicitALSRecommender(URM_train_validation)
als_recommender.fit(factors=1903, regularization=0.43781149051372426, iterations=10, use_gpu=False, **{"alpha": 10.365656056877015})

recommenders = [similarity_recommender, als_recommender]

alpha_linear = 0.7976651540039968
linear_comb_rec = GeneralizedLinearCoupleHybridRecommender(URM_train_validation, recommenders)
linear_comb_rec.fit(alpha_linear)

RP3betaRecommender: Similarity column 38121 (100.0%), 3736.36 column/sec. Elapsed time 10.20 sec


/Users/lucapagano/miniconda3/envs/RecSysPD/lib/python3.8/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 8 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/10 [00:00<?, ?it/s]

In [5]:
W_easeR = sps.load_npz("SimilarityMatrixes/easeR_best_W_TRAINVAL.npz")
easeR = ItemKNNCustomSimilarityRecommender(URM_train_validation)
easeR.fit(W_easeR)

In [6]:
from Recommenders.Recommender_utils import similarityMatrixTopK

W_easeR = similarityMatrixTopK(W_easeR, k=100, use_absolute_values=True)
W_easeR = W_easeR.tocsr()

easeR.fit(W_easeR)

In [7]:
import gc
gc.collect()

73

In [8]:
new_recommenders = [linear_comb_rec, easeR]

def objective_function(trial):
    alpha = trial.suggest_uniform('alpha', 0.5, 1.0)

    recommender = GeneralizedLinearCoupleHybridRecommender(URM_train_validation, new_recommenders)
    recommender.fit(alpha)
    
    result_dict, _ = evaluator_test.evaluateRecommender(recommender)
    return result_dict.loc[10]["MAP"]

class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame(columns = ["result"])
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        self.results_df = pd.concat([self.results_df, pd.DataFrame([hyperparam_dict])], ignore_index=True)
        self.results_df.to_csv("logs/Hybrid/hybrid_slim_rp3_IALS_EASERONLYLINEAR.csv", index = False)
        
        
import optuna

optuna_study = optuna.create_study(study_name="hybrid_slim_rp3_IALS_EASER", direction="maximize")
        
save_results = SaveResults()
        
optuna_study.optimize(objective_function,
                      callbacks=[save_results],
                      n_trials = 50
                      )

[I 2025-01-07 08:59:15,354] A new study created in memory with name: hybrid_slim_rp3_IALS_EASER
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_98289/3949591026.py:4: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform('alpha', 0.5, 1.0)


EvaluatorHoldout: Processed 35595 (100.0%) in 40.17 sec. Users per second: 886


[I 2025-01-07 08:59:55,744] Trial 0 finished with value: 0.061523046976900704 and parameters: {'alpha': 0.9665280542278493}. Best is trial 0 with value: 0.061523046976900704.
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_98289/3949591026.py:4: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform('alpha', 0.5, 1.0)


EvaluatorHoldout: Processed 35595 (100.0%) in 40.27 sec. Users per second: 884


[I 2025-01-07 09:00:36,038] Trial 1 finished with value: 0.061460670862902265 and parameters: {'alpha': 0.9105282791457197}. Best is trial 0 with value: 0.061523046976900704.
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_98289/3949591026.py:4: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform('alpha', 0.5, 1.0)


...

EvaluatorHoldout: Processed 35595 (100.0%) in 37.59 sec. Users per second: 947


[I 2025-01-07 09:12:43,045] Trial 20 finished with value: 0.06151192092700516 and parameters: {'alpha': 0.9587636889324728}. Best is trial 12 with value: 0.06155636269584651.
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_98289/3949591026.py:4: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform('alpha', 0.5, 1.0)


EvaluatorHoldout: Processed 35595 (100.0%) in 37.58 sec. Users per second: 947


[I 2025-01-07 09:13:20,657] Trial 21 finished with value: 0.06153801586186542 and parameters: {'alpha': 0.9907011626298935}. Best is trial 12 with value: 0.06155636269584651.
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_98289/3949591026.py:4: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform('alpha', 0.5, 1.0)
[W 2025-01-07 09:13:51,918] Trial 22 failed with parameters: {'alpha': 0.9987165291838465} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/lucapagano/miniconda3/envs/RecSysPD/lib/python3.8/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_98289/3949591026.py", line 9, in objective_function
    result_dict, _ = evaluator_test.evalua

KeyboardInterrupt: 